In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from scipy.stats import zscore
import math
from itertools import combinations

In [2]:
MSA_ZScore = pd.read_csv('/content/drive/MyDrive/03 Courses/02 Spring/2022 CUSP - Capstone project/05 Pipeline/01 MSA/01 Data/03_MSA_ZScore_ManualCheck.csv')
MSA_ZScore = MSA_ZScore[['GEO_ID','Total_housing_units','kitch_zscore','plumbing_zscore','age_zscore','income_zscore']]
MSA_ZScore

,GEO_ID,Total_housing_units,kitch_zscore,plumbing_zscore,age_zscore,income_zscore
0,1400000US48085030100,2242,0.513433,0.462763,0.576587,-0.265092
1,1400000US48085030201,1136,0.513433,0.462763,0.832801,0.183436
2,1400000US48085030202,1254,0.513433,0.462763,0.768747,0.840757
3,1400000US48085030203,6745,0.513433,0.462763,1.345228,0.640594
4,1400000US48085030301,2499,-0.201051,-0.447178,1.409282,0.872110
...,...,...,...,...,...,...
1306,1400000US48497150403,1519,0.086000,-0.573621,0.512533,-0.156722
1307,1400000US48497150500,2254,-0.494751,0.462763,-0.640429,-0.793669
1308,1400000US48497150601,3465,0.326053,-0.193497,0.320373,-0.408530
1309,1400000US48497150602,2681,0.362073,0.006056,0.896854,-0.168111


In [3]:
z_score_expand = MSA_ZScore.loc[MSA_ZScore.index.repeat(MSA_ZScore['Total_housing_units'])]
z_score_expand.reset_index(drop = True, inplace = True)
z_score_expand.drop(columns=['Total_housing_units','GEO_ID'], inplace = True)
z_score_expand

,kitch_zscore,plumbing_zscore,age_zscore,income_zscore
0,0.513433,0.462763,0.576587,-0.265092
1,0.513433,0.462763,0.576587,-0.265092
2,0.513433,0.462763,0.576587,-0.265092
3,0.513433,0.462763,0.576587,-0.265092
4,0.513433,0.462763,0.576587,-0.265092
...,...,...,...,...
2764973,-0.717628,-0.421657,0.704694,-0.508215
2764974,-0.717628,-0.421657,0.704694,-0.508215
2764975,-0.717628,-0.421657,0.704694,-0.508215
2764976,-0.717628,-0.421657,0.704694,-0.508215


In [4]:
z_score_arr = z_score_expand.to_numpy()
z_score_arr

array([[ 0.51343251,  0.46276292,  0.57658685, -0.26509239],
       [ 0.51343251,  0.46276292,  0.57658685, -0.26509239],
       [ 0.51343251,  0.46276292,  0.57658685, -0.26509239],
       ...,
       [-0.7176278 , -0.42165683,  0.70469375, -0.50821475],
       [-0.7176278 , -0.42165683,  0.70469375, -0.50821475],
       [-0.7176278 , -0.42165683,  0.70469375, -0.50821475]])

In [5]:
# PCA
pca_msa = PCA()
pca_msa.fit(z_score_arr)
print(pca_msa.components_)

[[-0.34277792 -0.38493289 -0.58819572 -0.62318197]
 [-0.62405696 -0.58455079  0.41019681  0.3171622 ]
 [ 0.69258854 -0.68155915  0.19045803 -0.13972824]
 [ 0.11566038 -0.21355132 -0.67044024  0.70109086]]


In [6]:
# EIGENVECTORS

kitch      =  abs(pca_msa.components_[0][0])
plumbing   =  abs(pca_msa.components_[0][1])
age        =  abs(pca_msa.components_[0][2])
income     =  abs(pca_msa.components_[0][3])


print('kitch      = {}'.format(kitch))
print('plumbing   = {}'.format(plumbing))
print('age        = {}'.format(age))
print('income     = {}'.format(income))

kitch      = 0.3427779165516183
plumbing   = 0.38493289239956274
age        = 0.588195718175507
income     = 0.6231819681228254


In [7]:
k = 1/(math.sqrt(age**2 + income**2 + kitch**2 + plumbing**2))
k

1.0

In [8]:
MSA_ZScore_PCA = MSA_ZScore.copy()
MSA_ZScore_PCA['PCA_weighted_score'] = (k*age*MSA_ZScore_PCA['age_zscore']) + (k*kitch*MSA_ZScore_PCA['kitch_zscore']) + (k*plumbing*MSA_ZScore_PCA['plumbing_zscore']) + (k*income*MSA_ZScore_PCA['income_zscore'])
MSA_ZScore_PCA

,GEO_ID,Total_housing_units,kitch_zscore,plumbing_zscore,age_zscore,income_zscore,PCA_weighted_score
0,1400000US48085030100,2242,0.513433,0.462763,0.576587,-0.265092,0.528071
1,1400000US48085030201,1136,0.513433,0.462763,0.832801,0.183436,0.958290
2,1400000US48085030202,1254,0.513433,0.462763,0.768747,0.840757,1.330244
3,1400000US48085030203,6745,0.513433,0.462763,1.345228,0.640594,1.544590
4,1400000US48085030301,2499,-0.201051,-0.447178,1.409282,0.872110,1.131367
...,...,...,...,...,...,...,...
1306,1400000US48497150403,1519,0.086000,-0.573621,0.512533,-0.156722,0.012477
1307,1400000US48497150500,2254,-0.494751,0.462763,-0.640429,-0.793669,-0.862755
1308,1400000US48497150601,3465,0.326053,-0.193497,0.320373,-0.408530,-0.028866
1309,1400000US48497150602,2681,0.362073,0.006056,0.896854,-0.168111,0.549204


In [9]:
# check if the weighted Z Score has zero sum
(MSA_ZScore_PCA['PCA_weighted_score'] * MSA_ZScore_PCA['Total_housing_units']).sum()

1.076841726899147e-09

In [10]:
from google.colab import files 
MSA_ZScore_PCA.to_csv('04_MSA_ZScore_PCA.csv', index=False) 
files.download('04_MSA_ZScore_PCA.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#PCA EXPLORATION

In [18]:
MSA_columns = list(z_score_expand.columns)
MSA_columns

['kitch_zscore', 'plumbing_zscore', 'age_zscore', 'income_zscore']

In [22]:
MSA_columns_combo = list(combinations(MSA_columns, 2))
MSA_columns_combo

[('kitch_zscore', 'plumbing_zscore'),
 ('kitch_zscore', 'age_zscore'),
 ('kitch_zscore', 'income_zscore'),
 ('plumbing_zscore', 'age_zscore'),
 ('plumbing_zscore', 'income_zscore'),
 ('age_zscore', 'income_zscore')]

In [23]:
for i in range(len(MSA_columns_combo)):
  test_list = list(MSA_columns_combo[i])
  MSA_ZScore_expand_sub = z_score_expand[test_list]
  MSA_ZScore_arr_sub = MSA_ZScore_expand_sub.to_numpy()
  pca_msa_sub = PCA()
  pca_msa_sub.fit(MSA_ZScore_arr_sub)
  var_1 = abs(pca_msa_sub.components_[0][0])
  var_2 = abs(pca_msa_sub.components_[0][1])
  print('{} = {}'.format(test_list[0],var_1))
  print('{} = {}'.format(test_list[1],var_2))

kitch_zscore = 0.7071067811865459
plumbing_zscore = 0.707106781186549
kitch_zscore = 0.7071067811865429
age_zscore = 0.7071067811865522
kitch_zscore = 0.7071067811865402
income_zscore = 0.7071067811865549
plumbing_zscore = 0.7071067811865357
age_zscore = 0.7071067811865595
plumbing_zscore = 0.7071067811865431
income_zscore = 0.7071067811865519
age_zscore = 0.7071067811865461
income_zscore = 0.707106781186549


In [12]:
sub_list = ['age_zscore','income_zscore']

In [13]:
z_score_expand_sub = z_score_expand[sub_list]
z_score_expand_sub

,age_zscore,income_zscore
0,0.576587,-0.265092
1,0.576587,-0.265092
2,0.576587,-0.265092
3,0.576587,-0.265092
4,0.576587,-0.265092
...,...,...
2764973,0.704694,-0.508215
2764974,0.704694,-0.508215
2764975,0.704694,-0.508215
2764976,0.704694,-0.508215


In [14]:
z_score_expand_sub_arr = z_score_expand_sub.to_numpy()
z_score_expand_sub_arr

array([[ 0.57658685, -0.26509239],
       [ 0.57658685, -0.26509239],
       [ 0.57658685, -0.26509239],
       ...,
       [ 0.70469375, -0.50821475],
       [ 0.70469375, -0.50821475],
       [ 0.70469375, -0.50821475]])

In [15]:
# PCA
pca_msa_sub = PCA()
pca_msa_sub.fit(z_score_expand_sub_arr)
print(pca_msa_sub.components_)

[[ 0.70710678  0.70710678]
 [-0.70710678  0.70710678]]


In [16]:
# EIGENVECTOR COMPONENTS

var_1 = abs(pca_msa_sub.components_[0][0])
var_2 = abs(pca_msa_sub.components_[0][1])

print('{} = {}'.format(sub_list[0],var_1))
print('{} = {}'.format(sub_list[1],var_2))

age_zscore = 0.7071067811865461
income_zscore = 0.707106781186549
